In [2]:
import pandas as pd
import numpy as np
import re  #regular expresion : used for searching the text in a document 
from nltk.corpus import stopwords # nltk: natural language toolkit -- stopwords: what ,a, the etc. 
from nltk.stem.porter import PorterStemmer # stemming remove prefix and suffix of the word and generate root word
#TfidfVectorizer : convert the text into feature vectors(can say numbers)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
print(stopwords.words('english')) # shows all stop words 

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [5]:
#-------------------------------------------------Data preprocessing ----------------------------------------------------

In [6]:
df=pd.read_csv('train.csv')
df.head() # 1 represent fake news 0 represent real news 

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.shape

(20800, 5)

In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
df=df.fillna('') #replacing null values with empty string 

In [10]:
# mering author name and title 
df['content']=df['author']+ ' ' +df['title']	

In [11]:
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [12]:
# separating the datalabel
x=df.drop(columns='label',axis=1)
y=df['label']

In [13]:
x

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [14]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [15]:
#stemming: process of reducing a word to its root word ex: raining-->rain
stem=PorterStemmer()

def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z],','',content) # ^a-z A-Z this will remove nuber and everythng which doesnot have alphabets from content and the space will be added inthe place of removed characters 
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")] # it remove stop words 
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content


In [16]:
df['content']=df['content'].apply(stemming)

In [17]:
print(df['content'])

0        darrel lucu hous dem aide: didn’t even see com...
1        daniel j. flynn flynn: hillari clinton, big wo...
2                  consortiumnews.com truth might get fire
3        jessica purkiss 15 civilian kill singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t.i.: trump ’poster child ...
20796    benjamin hoffman n.f.l. playoffs: schedule, ma...
20797    michael j. de la merc rachel abram macy’ said ...
20798    alex ansari nato, russia hold parallel exercis...
20799                         david swanson keep f-35 aliv
Name: content, Length: 20800, dtype: object


In [18]:
x=df['content'].values
y=df['label'].values

In [19]:
y.shape

(20800,)

In [20]:
# converting textual data to numeric data
vectorizer=TfidfVectorizer() # tf=term frequency idf= inverse document frequency : it basically count the no of times  a perticular word is repeating in a perticular text paragraph , so the repeatation tells the model that the word is impottant , 
                            # idf: some repeated words are not important so this function finds those words and gives values 
vectorizer.fit(x)
x=vectorizer.transform(x)


In [21]:
print(x) # text converted to vector 

  (0, 831)	0.3646500188253278
  (0, 3425)	0.33756896138985654
  (0, 3974)	0.2266469969205269
  (0, 4785)	0.33044571153796654
  (0, 5017)	0.2512923264945339
  (0, 5278)	0.27468869329117757
  (0, 6455)	0.21422587910261737
  (0, 8855)	0.20534182453318398
  (0, 9715)	0.22757176689298134
  (0, 10769)	0.26822209263186303
  (0, 11094)	0.33384522056560495
  (0, 16494)	0.23676064517956455
  (0, 19118)	0.27315635150958634
  (1, 2280)	0.29282032322324314
  (1, 2735)	0.15460262441734718
  (1, 3122)	0.38708204516205785
  (1, 3800)	0.19062864013183065
  (1, 4750)	0.26268912792483357
  (1, 7123)	0.7114899247272568
  (1, 8631)	0.19810516182474192
  (1, 20437)	0.299521873400654
  (2, 3955)	0.31669937181571
  (2, 4184)	0.4602325600950218
  (2, 6990)	0.39293503470255664
  (2, 7673)	0.34605253138342823
  :	:
  (20797, 8897)	0.21736692401926294
  (20797, 10453)	0.22258731789449554
  (20797, 11201)	0.36057430237452964
  (20797, 11788)	0.2945753459972408
  (20797, 11864)	0.17405416749037161
  (20797, 12705)	

In [22]:
x_train,x_test,y_train,y_test= train_test_split(x,y,stratify=y,test_size=0.2,random_state=42)

In [23]:
 model=LogisticRegression()

In [24]:
model.fit(x_train,y_train)

LogisticRegression()

In [25]:
# accuracy score for training 
x_pred=model.predict(x_train)
accuracy=accuracy_score(x_pred,y_train)
accuracy

0.9878004807692308

In [26]:
# accuracy score for text
x_pred=model.predict(x_test)
accuracy=accuracy_score(x_pred,y_test)
accuracy

0.975

In [27]:
# making prediction system 
x_new=x_test[0] #1st row xtest column 
pred=model.predict(x_new)
print(pred)

if pred==0:
    print('news is real')
else:
    print('news is fake')


[0]
news is real


In [28]:
print(y_test[0])

0
